# Data Simulation 

## Define Functions

In [22]:
import numpy as np
from random import choice

In [23]:
# Generate d-dimensional feature vector
def simulate_feature_vec(n, d, a=2):
    # Random states
    def get_states(random_state, low, high, size):
        rs = np.random.RandomState(random_state)
        states = rs.randint(low=low, high=high, size=size)
        return states
    states1 = get_states(random_state=42, low=0, high=100000, size=d)
    states2 = get_states(random_state=1028, low=0, high=100000, size=d)    
    # generate one sequence for raw covariance matrice
    def generate_seq(i):
        np.random.seed(states1[i])
        a = np.random.randn(d)
        np.random.seed(states2[i])
        b = np.random.randn(1)
        return a+b    
    # Generate random covariance matrice
    A = np.matrix([generate_seq(i) for i in range(d)])
    A = A*np.transpose(A)
    D_half = np.diag(np.diag(A)**(-0.5))
    C = D_half*A*D_half
    # Generate d-dimensional feature vector
    mean = np.zeros(d)
    cov = C
    x = np.random.multivariate_normal(mean, cov, n) 
    return x # shape (n,d)

In [30]:
# Generate potential outcomes
def simulate_y1(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    beta = np.random.uniform(-5,5,d)
    mu0 = np.dot(x, beta) +5*(x[:,0] > 0.5) 
    mu1 = mu0 + 8*(x[:,1] > 0.1)
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group  

In [31]:
# Generate potential outcomes
def simulate_y2(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    beta0 = np.random.uniform(1,30,d)
    beta1 = np.random.uniform(1,30,d)
    mu0 = np.dot(x, beta0)
    mu1 = np.dot(x, beta1)
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group  

In [32]:
# Generate potential outcomes
def simulate_y3(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    effect = 4 / (1+np.exp(-12 * ((x[:,0]-1)/2))) * (1+np.exp(-12 * ((x[:,1]-1)/2)))
    mu1 = 0.5 * effect
    mu0 = -mu1
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group  

In [33]:
# Generate potential outcomes
def simulate_y4(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    beta = np.random.uniform(1,30,d)
    mu0 = np.dot(x, beta)
    mu1 = mu0
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group  

In [34]:
# Generate potential outcomes
def simulate_y5(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    beta = np.random.uniform(-15,15,d)
    beta_m_dim = min(d,5)
    beta_m = beta[0:beta_m_dim]
    mu0 = np.zeros(n)
    mu0[x[:,19] < -0.4] = np.dot(x[x[:,19] < 0.-0.4][:,0:beta_m_dim],beta_m)
    mu0[(x[:,19] < 0.4) & (x[:,19] >= -0.4)] = np.dot(x[(x[:,19] < 0.4) & (x[:,19] >= -0.4)][:,beta_m_dim:(2*beta_m_dim)],beta_m)
    mu0[x[:,19] >= 0.4] = np.dot(x[x[:,19] >= 0.4][:,2*beta_m_dim:3*beta_m_dim],beta_m)
    mu0 = np.dot(x, beta)
    mu1 = mu0
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group  

In [35]:
# Generate potential outcomes
def simulate_y6(n,d):
    # Generate error term matrice
    e1 = np.random.randn(n)
    e0 = np.random.randn(n)
    # Generate mu
    mu0 = 2 * x[:,0] - 1
    mu1 = mu0
    # Calculate y
    y1 = mu1 + e1
    y0 = mu0 + e0
    return y1,y0 # y1:treatement group #y0:control group 

In [36]:
# Generate treatment assignment
def simulate_assignment(propensity):
    w = np.random.binomial(1,propensity,n)
    return w

# Different propensity scores
def simulate_assignment_y6(n):
    prop_list = 1/4 * (1+ np.random.beta(2,4,n))
    w = np.random.binomial(1,prop_list,n)
    return w

In [37]:
# Generate assigned y
def generate_assigned_y(y1,y0,w):
    y_obs = y1*w - y0*(w-1)
    return y_obs

In [38]:
# Calculate effect
def calculate_effect(y1, y0):
    eff_act = y1 - y0
    return eff_act

## Give parameters and generate data

In [40]:
# Simulation_1_a: Unbalanced
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.01 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y0(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [43]:
# Simulation_1_b: Unbalanced
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.5 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y1(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [14]:
# Simulation_2
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.5 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y2(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [15]:
# Simulation_3
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.5 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y3(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [16]:
# Simulation_4
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentr ation of covariance matrix 1, -1,0
propensity = 0.5 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y4(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [17]:
# Simulation_5
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.5 # propensity score of Bernouli distribution of Y

# Generate data
x = simulate_feature_vec(n,d) 
y1, y0 = simulate_y5(n,d)
w = simulate_assignment(propensity)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [18]:
# Simulation_6
# Parameters
n = 500000 # number of samples 
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0

# Generate Data
x = np.random.uniform(0,1,(n,d)) 
y1, y0 = simulate_y6(n,d)
w = simulate_assignment_y6(n)
y_obs = generate_assigned_y(y1,y0,w)
eff_act = calculate_effect(y1, y0)

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from bartpy.sklearnmodel import SklearnModel

train_lens = [5000, 10000, 25000, 50000 100000, 200000]
d = 20 # dimentionality of Xi
a = 2 # Concentration of covariance matrix 1, -1,0
propensity = 0.01 # propensity score of Bernouli distribution of Y

## S-BART

In [45]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    Y = np.array([y0[i] if w[i] else y1[i] for i in range(len(w))]).reshape((n,1)) # get Y for training
    W = w.reshape((w.shape[0], 1))
    X = np.concatenate((x, W),axis=1)
    
    x_train, y_train = X, Y

    S_bart = SklearnModel()
    S_bart.fit(x_train, y_train)
    models.append(S_bart)
    
    w0 = np.zeros((w.shape[0], 1))
    w1 = np.ones((w.shape[0], 1))
    X0 = np.concatenate((x, w0),axis=1)
    X1 = np.concatenate((x, w1),axis=1)
    
    y_true = y1 - y0
    y0_pred = S_bart.predict(x_train)
    y1_pred = S_bart.predict(x_train)
    y_pred = y1_pred - y0_pred
    mse_train = mean_squared_error(y_true, y_pred)
    MSE.append(mse_train)

C:\Users\yupei\AppData\anaconda3\lib\site-packages\joblib\disk.py:122: UserWarning: Unable to delete folder C:\Users\yupei\AppData\Local\Temp\joblib_memmapping_folder_15156_2281746886 after 5 tentatives.
  .format(folder_path, RM_SUBDIRS_N_RETRY))


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\yupei\\AppData\\Local\\Temp\\joblib_memmapping_folder_15156_2281746886\\15156-2543474100096-b8086ff43e4a4ca88cadc6b53cc13d1d.pkl'

In [ ]:
MSE

## S-RF

In [46]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    Y = np.array([y0[i] if w[i] else y1[i] for i in range(len(w))]).reshape((n,1)) # get Y for training
    W = w.reshape((w.shape[0], 1))
    X = np.concatenate((x, W),axis=1)
    
    x_train, y_train = X, Y

    S_rf = RandomForestRegressor(random_state=123, n_estimators=100)
    S_rf.fit(x_train, y_train)
    models.append(S_rf)
    
    w0 = np.zeros((w.shape[0], 1))
    w1 = np.ones((w.shape[0], 1))
    X0 = np.concatenate((x, w0),axis=1)
    X1 = np.concatenate((x, w1),axis=1)
    
    y_true = y1 - y0
    y0_pred = S_rf.predict(x_train)
    y1_pred = S_rf.predict(x_train)
    y_pred = y1_pred - y0_pred
    mse_train = mean_squared_error(y_true, y_pred)
    MSE.append(mse_train)

C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\yupei\AppData\anaconda3\lib\site-packages\ipykernel_launcher.py

In [47]:
MSE

[31.68189511573245,
 31.470518600739002,
 31.720425264119623,
 31.5131149796114,
 31.478653786350517]

## T-BART

In [ ]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    X_0 = np.array([x[i] for i in range(len(w)) if not w[i]])
    X_1 = np.array([x[i] for i in range(len(w)) if w[i]])

    Y_0 = np.array([y0[i] for i in range(len(w)) if not w[i]])
    Y_1 = np.array([y1[i] for i in range(len(w)) if  w[i]])
    
    T_bart_0 = SklearnModel()
    T_bart_1 = SklearnModel()
    
    T_bart_0.fit(X_0, Y_0)
    T_bart_1.fit(X_1, Y_1)
    
    y0_pred = T_bart_0.predict(x)
    y1_pred = T_bart_1.predict(x)
    y_pred = y1_pred - y0_pred
    y_true = y1 - y0
    
    mse = mean_squared_error(y_pred, y_true)
    models.append((T_rt_0, T_rt_1))
    MSE.append(mse)

In [ ]:
MSE

## T-RF

In [23]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    X_0 = np.array([x[i] for i in range(len(w)) if not w[i]])
    X_1 = np.array([x[i] for i in range(len(w)) if w[i]])

    Y_0 = np.array([y0[i] for i in range(len(w)) if not w[i]])
    Y_1 = np.array([y1[i] for i in range(len(w)) if  w[i]])
    
    T_rf_0 = RandomForestRegressor(random_state=123, n_estimators=100)
    T_rf_1 = RandomForestRegressor(random_state=123, n_estimators=100)
    
    T_rf_0.fit(X_0, Y_0)
    T_rf_1.fit(X_1, Y_1)
    
    y0_pred = T_rf_0.predict(x)
    y1_pred = T_rf_1.predict(x)
    y_pred = y1_pred - y0_pred
    y_true = y1 - y0
    
    mse = mean_squared_error(y_pred, y_true)
    models.append((T_rf_0, T_rf_1))
    MSE.append(mse)

In [24]:
MSE

[30.957384283878458,
 39.82619269509149,
 35.590800811454805,
 12.3637527644378,
 16.933560936625103]

## X-BART

In [ ]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    X_0 = np.array([x[i] for i in range(len(w)) if not w[i]])
    X_1 = np.array([x[i] for i in range(len(w)) if w[i]])

    Y_0 = np.array([y0[i] for i in range(len(w)) if not w[i]])
    Y_1 = np.array([y1[i] for i in range(len(w)) if  w[i]])
    
    X_bart_0 = SklearnModel()
    X_bart_1 = SklearnModel()
    
    X_bart_0.fit(X_0, Y_0)
    X_bart_1.fit(X_1, Y_1)
    
    D_1 = Y_1 - X_rf_0.predict(X_1)
    D_0 = X_bart_1.predict(X_0) - Y_0

    X_bart_3 = SklearnModel()
    X_bart_4 = SklearnModel()
    
    X_bart_3.fit(X_1, D_1)
    X_bart_4.fit(X_0, D_0)
    
    
    y1_pred = X_bart_3.predict(x)
    y0_pred = X_bart_4.predict(x)
    
    y_pred = propensity * y0_pred + (1 - propensity) * y1_pred
    y_true = y1 - y0
    
    mse = mean_squared_error(y_pred, y_true)
    models.append((X_rf_0, X_rf_1, X_rf_3, X_rf_4))
    MSE.append(mse)

In [39]:
MSE

[31.68189511573245]

## X-RF

In [25]:
models = []
MSE = []
for n in train_lens:
    
    x = simulate_feature_vec(n,d) 
    y1, y0 = simulate_y1(n,d)
    w = simulate_assignment(propensity)
    y_obs = generate_assigned_y(y1,y0,w)
    eff_act = calculate_effect(y1, y0)
    
    X_0 = np.array([x[i] for i in range(len(w)) if not w[i]])
    X_1 = np.array([x[i] for i in range(len(w)) if w[i]])

    Y_0 = np.array([y0[i] for i in range(len(w)) if not w[i]])
    Y_1 = np.array([y1[i] for i in range(len(w)) if  w[i]])
    
    X_rf_0 = RandomForestRegressor(random_state=123, n_estimators=100)
    X_rf_1 = RandomForestRegressor(random_state=123, n_estimators=100)
    
    X_rf_0.fit(X_0, Y_0)
    X_rf_1.fit(X_1, Y_1)
    
    D_1 = Y_1 - X_rf_0.predict(X_1)
    D_0 = X_rf_1.predict(X_0) - Y_0

    X_rf_3 = RandomForestRegressor(random_state=123, n_estimators=100)
    X_rf_4 = RandomForestRegressor(random_state=123, n_estimators=100)
    
    X_rf_3.fit(X_1, D_1)
    X_rf_4.fit(X_0, D_0)
    
    
    y1_pred = X_rf_3.predict(x)
    y0_pred = X_rf_4.predict(x)
    
    y_pred = propensity * y0_pred + (1 - propensity) * y1_pred
    y_true = y1 - y0
    
    mse = mean_squared_error(y_pred, y_true)
    models.append((X_rf_0, X_rf_1, X_rf_3, X_rf_4))
    MSE.append(mse)

MemoryError: could not allocate 7340032 bytes

In [ ]:
MSE